In [4]:
import cv2
import numpy as np
import multiprocessing
import signal

In [5]:
class FaceAndEyeDetection(object):
    def __init__(self, video_path, face_model_path, eye_model_path):
        self.video_path = video_path
        self.face_cascade_model = cv2.CascadeClassifier(face_model_path)
        self.eye_model_model = cv2.CascadeClassifier(eye_model_path)

    def detect_faces(self, gray):
        faces = self.face_cascade_model.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for (x, y, w, h) in faces:
            cv2.rectangle(self.image, (x, y), (x+w, y+h), (255, 255, 0), 2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = self.image[y:y+h, x:x+w]

            eyes = self.eye_model_model.detectMultiScale(roi_gray)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 127, 255), 2)

    def process_frame(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.detect_faces(gray)

    def run(self):

        cap = cv2.VideoCapture(self.video_path)
        while True:
            ret, self.image = cap.read()

            if not ret:
                break

            self.process_frame(self.image)
            cv2.imshow('image', self.image)
            k = cv2.waitKey(1) & 0xff
            if k == 27:  # Press 'Esc' to exit
                break

        cap.release()
        cv2.destroyAllWindows()


def wrapper(video_path, face_model_path, eye_model_path):
    model = FaceAndEyeDetection(video_path, face_model_path, eye_model_path)
    model.run()


def signal_handler(signum, frame):
    print("Received Signal. Exiting gracefully.")
    exit(0)

In [6]:
if __name__ == '__main__':
    # very useful when this is run as a standalone .py file
    signal.signal(signal.SIGINT, signal_handler)

    face_model_path = './models/haarcascade_frontalface_default.xml'
    eye_model_path = './models/haarcascade_eye.xml'

    video_files = ['./test-videos/video-1.mp4', './test-videos/video-2.mp4', './test-videos/video-3.mp4', './test-videos/video-4.mp4']

    jobs = []
    for video_path in video_files:
        p = multiprocessing.Process(target=wrapper, args=(video_path, face_model_path, eye_model_path))
        jobs.append(p)
        p.start()

    for job in jobs:
        job.join()
